In [1]:


import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

import torch, sys
import json
import pandas as pd
import numpy as np


from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from transformers.trainer_pt_utils import LabelSmoother
from datasets import load_dataset, Dataset
from pprint import pprint

In [2]:
from accelerate.utils import write_basic_config

write_basic_config??

write_basic_config(mixed_precision="",  # 是否使用混合精度， 默认是no，可以设置：no， fp16, bf16
                  save_location="",  # accelerate的配置文件保存地址，默认：/root/.cache/huggingface/accelerate/default_config.yaml
                  use_xpu=False)


Configuration already exists at , will not override. Run `accelerate config` manually or pass a different `save_location`.


False

Signature:
write_basic_config(
    mixed_precision='no',
    save_location: str = '/root/.cache/huggingface/accelerate/default_config.yaml',
    use_xpu: bool = False,
)
Source:   
def write_basic_config(mixed_precision="no", save_location: str = default_json_config_file, use_xpu: bool = False):
    """
    Creates and saves a basic cluster config to be used on a local machine with potentially multiple GPUs. Will also
    set CPU if it is a CPU-only machine.

    Args:
        mixed_precision (`str`, *optional*, defaults to "no"):
            Mixed Precision to use. Should be one of "no", "fp16", or "bf16"
        save_location (`str`, *optional*, defaults to `default_json_config_file`):
            Optional custom save location. Should be passed to `--config_file` when using `accelerate launch`. Default
            location is inside the huggingface cache folder (`~/.cache/huggingface`) but can be overriden by setting
            the `HF_HOME` environmental variable, followed by `acce

In [3]:

train_json_path = "/root/autodl-tmp/ie/yayi_uie_sft_data/train_sft.jsonl"

test_json_path = "/root/autodl-tmp/ie/yayi_uie_sft_data/test_sft.jsonl"



def load_dataset_from_disk(json_path):


    pass



dataset = load_dataset("json", data_files=test_json_path, split="train")
# print(dataset)
pprint(dataset[0])





{'Dataset': 'DuEE1.0',
 'Task': 'CEE',
 'input': '',
 'instruction': '文本: 男子以帮找工作名义骗钱被警方抓获 \n'
                "【事件抽取】事件类型列表=['财经/交易-出售/收购', '财经/交易-跌停', '财经/交易-加息', "
                "'财经/交易-降价', '财经/交易-降息', '财经/交易-融资', '财经/交易-上市', '财经/交易-涨价', "
                "'财经/交易-涨停', '产品行为-发布', '产品行为-获奖', '产品行为-上映', '产品行为-下架', "
                "'产品行为-召回', '交往-道歉', '交往-点赞', '交往-感谢', '交往-会见', '交往-探班', "
                "'竞赛行为-夺冠', '竞赛行为-晋级', '竞赛行为-禁赛', '竞赛行为-胜负', '竞赛行为-退赛', "
                "'竞赛行为-退役', '人生-产子/女', '人生-出轨', '人生-订婚', '人生-分手', '人生-怀孕', "
                "'人生-婚礼', '人生-结婚', '人生-离婚', '人生-庆生', '人生-求婚', '人生-失联', "
                "'人生-死亡', '司法行为-罚款', '司法行为-拘捕', '司法行为-举报', '司法行为-开庭', "
                "'司法行为-立案', '司法行为-起诉', '司法行为-入狱', '司法行为-约谈', '灾害/意外-爆炸', "
                "'灾害/意外-车祸', '灾害/意外-地震', '灾害/意外-洪灾', '灾害/意外-起火', '灾害/意外-坍/垮塌', "
                "'灾害/意外-袭击', '灾害/意外-坠机', '组织关系-裁员', '组织关系-辞/离职', '组织关系-加盟', "
                "'组织关系-解雇', '组织关系-解散', '组织关系-解约', '组织关系-停职', '组织关系-退出', "


In [4]:
"""初始化一些参数"""
# model_name = "Qwen/Qwen2-7B-Instruct"
# cache_dir = "/root/autodl-tmp/qwen2-7b-instruct"

model_name = "Qwen/Qwen1.5-7B-Chat"
cache_dir = "/root/autodl-tmp/Qwen1.5-7B-Chat/"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir, trust_remote_code=True, padding="right")
max_length = 4098
max_length = 1232 + 256
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=cache_dir, device_map="auto", torch_dtype=torch.bfloat16)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
"""测试tokenizer"""

print(tokenizer("稀土开发者"), tokenizer.decode([113129]))

# one_example = dataset[0]
# print(type(one_example))
# pprint(one_example)


"""
测试对话模板
"""
prompt = ""

messages = [
    {"role": "assistant", "content": "You are a helpful assistant."},
    {"role": "user", "content": "我要退货"},
    {"role": "system", "content": "请提供单号"},
    {"role": "user", "content": "123456"},
    {"role": "assistant", "content": "已为您退货完成"}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print(text)

print(tokenizer.chat_template)


{'input_ids': [113921, 113129], 'attention_mask': [1, 1]} 开发者
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>assistant
You are a helpful assistant.<|im_end|>
<|im_start|>user
我要退货<|im_end|>
<|im_start|>system
请提供单号<|im_end|>
<|im_start|>user
123456<|im_end|>
<|im_start|>assistant
已为您退货完成<|im_end|>
<|im_start|>assistant

{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}{{ '<|im_start|>system
You are a helpful assistant.<|im_end|>
' }}{% endif %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_end|>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}


In [6]:



# print(one_example["instruction"])


from typing import Dict, List

# 参考qwen2官方参考的finetuning代码，发现是错误的
def process_fun_error(example: Dict):
    """自己实现instruction和response的区分"""
    system_prompt = "你是一个文本信息抽取领域的专家，现在需要你从用户user的给定文本中任务类型完成对应的信息抽取任务"
    instruction = example["instruction"]
    response = example["label"]
    messages = [{"role": "user", "content": instruction}, {"role": "system", "content": labels}]
    tokenized_message_ids = tokenizer.apply_chat_template(messages, 
                                                      tokenize=True, 
                                                      chat_template=tokenizer.chat_template,
                                                      add_generation_prompt=False, 
                                                      padding="max_length", 
                                                      max_length=8192, 
                                                      truncation=True)  # # list[int]
    # print(tokenizer.decode(tokenized_message_ids))
    input_ids = torch.tensor(tokenized_message_ids, dtype=torch.int)
    target_ids = input_ids.clone()
    target_ids[target_ids==tokenizer.pad_token_id] = LabelSmoother.ignore_index  # -100
    attenion_mask = input_ids.ne(tokenizer.pad_token_id).int()   # boolean转int类型
    return dict(input_ids=input_ids, attenion_mask=attenion_mask, target_ids=target_ids)


"""
对数据的预处理操作，根据不同的要求有不同的处理方法：
1、根据instruction 和 response 分别进行tokenize化，然后进行组装，根据设置的最大长度进行阶段

2、根据instruction 和 response 先进行拼接，然后tokeniz化，根据设置的最大长度进行拼接。（trl中的DataCollatorForOnlyCompletion就是这样操作的）



上述两种方法本质上没有特殊的区别：但是在最大长度不能包括全部数据上时，就会出现response不全的情况，对于LLM大模型抽取的任务来说，就会产生不一致的结果（特别是要求输出为json的任务）


"""

def process_fun_qwen15(example):
    
    instruction = example["instruction"]
    response = example["label"]
    # messages = [{"role": "user", "content": instruction}, {"role": "system", "content": labels}]
    # tokenized_message_ids = tokenizer.apply_chat_template(messages, 
    #                                                   tokenize=True, 
    #                                                   chat_template=tokenizer.chat_template,
    #                                                   add_generation_prompt=False, 
    #                                                   padding="max_length", 
    #                                                   max_length=8192, 
    #                                                   truncation=True)  # # list[int]
    
    template = f"<|im_start|>system\n你是一个文本信息抽取领域的专家，现在需要你从用户user的给定文本中任务类型完成对应的信息抽取任务<|im_end|>\n<|im_start|>user\n{instruction }<|im_end|>\n<|im_start|>assistant\n"
    tokenized_instruction = tokenizer(template, add_special_tokens=False, truncation=True)
    tokenized_response = tokenizer(response, add_special_tokens=False, truncation=True)
    # 组装了输入和输出（这是根据id进行组装的，更好的处理方式是：）
    input_ids = tokenized_instruction["input_ids"] + tokenized_response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = tokenized_instruction["attention_mask"] + tokenized_response["attention_mask"] + [1]
    labels = [-100] * len(tokenized_instruction["input_ids"]) + tokenized_response["input_ids"] + [tokenizer.pad_token_id]

    if len(input_ids) > max_length:
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


# example_inputs = process_fun(one_example)
# tt = example_inputs["target_ids"]
# print("target_padding的大小：", tt.shape)
# print(tt,tokenizer.pad_token_id, LabelSmoother.ignore_index)

# ta = tt[tt != -100]
# print("target的大小：", ta.shape)
# # print(tokenizer.decode(ta))

    
# ii = example_inputs["input_ids"]
# print("解码如下--------------------------：\n", tokenizer.decode(ii))

def collate_pad_fn(examples):
    return tokenizer.pad(examples, return_tensors="pt", padding=True, max_length=max_length)



tokenized_dataset = dataset.map(process_fun_qwen15, remove_columns=dataset.column_names, batched=False)

print(tokenized_dataset)

print(len(tokenized_dataset[:2]["input_ids"]))   # 长度为2
# print(tokenized_dataset[:2])
print("打印前5个的长度")

for index, example in enumerate(tokenized_dataset):
    if index > 6:
        break
    else:
        print(len(example["input_ids"]), len(example["attention_mask"]), len(example["labels"]))

Map:   0%|          | 0/5450 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5450
})
2
打印前5个的长度
1101 1101 1101
1110 1110 1110
1110 1110 1110
1117 1117 1117
1231 1231 1231
1139 1139 1139
1121 1121 1121


In [15]:
"""
测试datasets.load_dataset产生的数据集和torch.utils.data.DataLoader的混合使用
参考： https://huggingface.co/docs/datasets/use_with_pytorch
"""

# tokenized_dataset_torch = tokenized_dataset.with_format("torch")
# tokenized_dataset_torch = 
tokenized_dataset.set_format("torch")
tokenized_dataset_torch = tokenized_dataset
print(tokenized_dataset_torch[0])
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding, DataCollatorForSeq2Seq

print(tokenized_dataset_torch)
from torch.utils.data import Dataset as TorchDataset


# dataloader = DataLoader(tokenized_dataset_torch, batch_size=1, shuffle=False, collate_fn=DataCollatorWithPadding(tokenizer=tokenizer,pad_to_multiple_of=8),)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True, max_length=1024)
print("max_length: ", max_length)
dataloader = DataLoader(tokenized_dataset_torch, batch_size=32, shuffle=True, collate_fn=data_collator)


max_input_length = 0

for i, batch in enumerate(dataloader):
    if batch["input_ids"].shape[1] > max_input_length:
        max_input_length = batch["input_ids"].shape[1]
    if i > 6:
        # break
        continue
    else:
        print(batch["input_ids"].shape)

print("max_input_length: ", max_input_length)


{'input_ids': tensor([151644,   8948,    198,  ...,  30975,     60, 151643]), 'attention_mask': tensor([1, 1, 1,  ..., 1, 1, 1]), 'labels': tensor([  -100,   -100,   -100,  ...,  30975,     60, 151643])}
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5450
})
max_length:  1488
torch.Size([32, 1488])
torch.Size([32, 744])
torch.Size([32, 1197])
torch.Size([32, 1488])
torch.Size([32, 844])


/root/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


torch.Size([32, 634])
torch.Size([32, 1309])
max_input_length:  1488


In [ ]:



batch_inputs ={"input_ids":torch.tensor([[1, 2], [3, 4, 5]]), "attention_mask": torch.tensor([[1,1], [1,1,1]])}

print(tokenizer.pad(batch_inputs, padding=True, max_length=4))


batch_inputs = ["我去", "我去上海", "我要去上海了", "我明天去上海", ""]





In [ ]:

# 定义Lora


from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(task_type=TaskType.CAUSAL_LM, target_modules=["q_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
                        inference_mode=False,lora_alpha=32, r=8, lora_dropout=0.1)


# gradient_checkpoint=True, 必须设置: model.enable_input_require_grads()

gradient_checkpointing = True
if gradient_checkpointing:
    model.enable_input_require_grads()

model = get_peft_model(model=model, peft_config=lora_config)
print("模型可训练参数：", model.print_trainable_parameters())
args = TrainingArguments(output_dir="/root/autodl-tmp/yayi_output/",
                         per_device_eval_batch_size=1, gradient_accumulation_steps=1, 
                         logging_dir="/root/autodl-tmp/yayi_output/logging",
                         logging_steps=1, 
                         learning_rate=3e-5, 
                         save_steps=50, save_total_limit=1,
                         num_train_epochs=3, 
                         gradient_checkpointing=gradient_checkpointing)


trainer = Trainer(model=model, args=args, 
                  data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True, label_pad_token_id=-100),
                  train_dataset=tokenized_dataset)


trainer.train()



In [ ]:
# """
# 参考： https://github.com/QwenLM/Qwen2/blob/main/examples/sft/finetune.py#L144
# """

# from transformers import PreTrainedTokenizer
# import transformers

# TEMPLATE = "{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}{{ '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n' }}{% endif %}{{'<|im_start|>' + message['role'] + '\n' + message['content']}}{% if loop.last %}{{ '<|im_end|>'}}{% else %}{{ '<|im_end|>\n' }}{% endif %}{% endfor %}"



# def preprocess(
#     messages,
#     tokenizer: transformers.PreTrainedTokenizer,
#     max_len: int,
# ) -> Dict:
#     """Preprocesses the data for supervised fine-tuning."""

#     texts = []
#     for i, msg in enumerate(messages):
#         texts.append(
#             tokenizer.apply_chat_template(
#                 msg,
#                 chat_template=TEMPLATE,
#                 tokenize=True,
#                 add_generation_prompt=False,
#                 padding="max_length",
#                 max_length=max_len,
#                 truncation=True,
#             )
#         )
#     input_ids = torch.tensor(texts, dtype=torch.int)
#     target_ids = input_ids.clone()
#     target_ids[target_ids == tokenizer.pad_token_id] = LabelSmoother.ignore_index
#     attention_mask = input_ids.ne(tokenizer.pad_token_id)

#     return dict(
#         input_ids=input_ids, target_ids=target_ids, attention_mask=attention_mask
#     )




# # pprint(one_example)

# one_messages = [{"role": "user", "content": one_example["instruction"]}, {"role": "system", "content": one_example["label"]}]
# # print(one_messages)


# ins = preprocess([one_messages], tokenizer, max_len=8192)

# ins_ids = ins["input_ids"][0]
# # print(tokenizer.decode(ins_ids[ins_ids!=151643151643]))
# print(ins["attention_mask"][0])
